<a href="https://colab.research.google.com/github/lala991204/AI-based-financial-data-analysis/blob/master/market_data/Entropy_using_market%E2%80%99s_volume_and_anomaly_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import math
from sklearn.preprocessing import MinMaxScaler


## entropy process (long time)
def entr(min_chart):
    X= min_chart['volume'].values.reshape(-1,1)  
    scaler = MinMaxScaler(feature_range=(0, 5000))   
    scaler = scaler.fit(X)
    X_scaled = scaler.transform(X)
    
    vol1 = []
    for i in range(len(X_scaled)):
        vol1.append(int(X_scaled[i][0]))
    
    vol = list(set(vol1))
    vol_max = max(vol)  
    adj_matrix = np.zeros((vol_max+1, vol_max+1))
    
    for i in range(len(vol1)-1):
        adj_matrix[vol1[i]][vol1[i+1]] = 1
    
    for i in range(vol_max+1):     ## loop count 방지
        if adj_matrix[i][i] != 0:
            adj_matrix[i][i] = 0
    
    for i in range(vol_max+1):     ## 중복 count 방지
        for j in range(vol_max+1):
            if i != j:
                if adj_matrix[i][j] != 0 and adj_matrix[i][j] != 1:
                    adj_matrix[i][j] = 1
    
    in_degree = np.zeros(vol_max+1)
    out_degree = np.zeros(vol_max+1)
    
    for i in range(0, vol_max+1):
        for j in range(0, vol_max+1):
            in_degree[i] = in_degree[i] + adj_matrix[j][i]
            out_degree[i] = out_degree[i] + adj_matrix[i][j]

    trans_matrix = np.zeros((vol_max+1, vol_max+1))

    for i in range(vol_max+1):
        for j in range(vol_max+1):
            if (adj_matrix[i][j] == 1):
                trans_matrix[i][j] = adj_matrix[i][j] / out_degree[i]

    iden = np.eye(vol_max+1, vol_max+1)  # identity matrix
    one_s1 = np.ones(vol_max+1)
    one_s = [list(one_s1)]
    zero_s = np.zeros(vol_max+1)
    zeros_one = [list(np.append(zero_s, 1))]

    A=np.append(trans_matrix.T - iden,one_s,axis=0)
    b=np.array(zeros_one).T
    perron_vector = np.linalg.solve(np.matmul(A.T, A), np.matmul(A.T, b))
    
    ent = 0
    for i in range(vol_max+1):
        for j in range(vol_max+1):
            if (trans_matrix[i][j] > 0):
                ent=ent-(perron_vector[i][0]*(trans_matrix[i][j])*math.log(trans_matrix[i][j]))
    
    return ent



## 날짜 생성
date = pd.date_range(start='2021-01-01', end='2021-12-13')
result_df = pd.DataFrame(index=date)

## 데이터 로드
chart=pd.read_csv('OKEx_SWAP_OHLCV_1m_btc_usdt_swap_20210101-20211213.csv')

## Nan 값 처리
chart = chart.fillna(method='ffill')



##***********************close, entropy**************************

close=[]
for i in range(len(date)):
    close.append(chart['close'][1440*i:1440*(i+1)].values.mean())
result_df['close'] = close

entropy=[]
for i in tqdm(range(len(date))):
    entropy.append(entr(chart[1440*i:1440*(i+1)]))      ##　약 9시간 소요 (347개 날짜)

## entropy 값이 비교적 작게 나오므로 스케일링으로 조정.
scaler = MinMaxScaler(feature_range=(0,100))
scaled_entropy = scaler.fit_transform(np.array(entropy).reshape(-1,1))
result_df['entropy'] = scaled_entropy


##********************change_close, change_entropy**************************

change_entropy=[]
for i in range(len(result_df)-1):
    change_entropy.append(result_df['entropy'][i+1] - result_df['entropy'][i])
change_entropy.insert(0,0)
result_df['change_entropy'] = change_entropy


change_close=[]
for i in range(len(result_df)-1):
    change_close.append(result_df['close'][i+1] - result_df['close'][i])
change_close.insert(0,0)
result_df['change_close'] = change_close
## ************** percent_close *********************  
## 폭락/폭등 비율 나타냄(백분율)

result_df['percent_close'] = 0
for i in range(len(result_df)-1):
    result_df['percent_close'][i+1] =
 ((result_df['close'][i+1] - result_df['close'][i])/result_df['close'][i])*100


##**********decreasing_close, decreasing_entropy***********

result_df['decreasing_entropy'] = 0  
for i in range(len(result_df)):
    if result_df['change_entropy'][i] < 0:
       result_df['decreasing_entropy'][i] = 1

result_df['decreasing_close'] = 0   
for i in range(len(result_df)):
    if result_df['change_close'][i] < 0:
       result_df['decreasing_close'][i] = 1


##**************cusum_change_entropy***************
result_df['cusum_change_entropy'] = 0        ## 누적해가는 과정 나타내줌
for i in range(len(result_df)-1):
       if result_df['change_entropy'][i] < 0 and 
result_df['change_entropy'][i+1] < 0 and 
result_df['cusum_change_entropy'][i] == 0:
          result_df['cusum_change_entropy'][i+1] = 
result_df['change_entropy'][i] + result_df['change_entropy'][i+1]
          result_df['cusum_change_entropy'][i] = result_df['change_entropy'][i]        
       elif result_df['change_entropy'][i] > 0 and 
result_df['change_entropy'][i+1] > 0 and 
result_df['cusum_change_entropy'][i] == 0:
          result_df['cusum_change_entropy'][i+1] = 
result_df['change_entropy'][i] + result_df['change_entropy'][i+1]
          result_df['cusum_change_entropy'][i] = result_df['change_entropy'][i]      
       elif result_df['change_entropy'][i] < 0 and 
result_df['change_entropy'][i+1] < 0 
and result_df['cusum_change_entropy'][i] != 0:
          result_df['cusum_change_entropy'][i+1] = 
result_df['cusum_change_entropy'][i] + result_df['change_entropy'][i+1]
       elif result_df['change_entropy'][i] > 0 and 
result_df['change_entropy'][i+1] > 0 and 
result_df['cusum_change_entropy'][i] != 0:
          result_df['cusum_change_entropy'][i+1] = 
result_df['cusum_change_entropy'][i] + result_df['change_entropy'][i+1]
          
       


## 나머지 값은 그대로 채워줌
for i in range(len(result_df)):
    if result_df['cusum_change_entropy'][i] == 0:
        result_df['cusum_change_entropy'][i] = result_df['change_entropy'][i]

##*****************risk_detection**************************

result_df['risk_detection'] = 0
for i in range(len(result_df)-3):
    a= 0
    if result_df['cusum_change_entropy'][i] < 0:
        a = result_df['cusum_change_entropy'][i]
        if result_df['cusum_change_entropy'][i+1] > 0 and
 abs(a) + 15 < result_df['cusum_change_entropy'][i+1] and
 result_df['risk_detection'][i] == 0:       
            result_df['risk_detection'][i+1] = 1 
        elif result_df['cusum_change_entropy'][i+2] > 0 and 
abs(a) + 15 < result_df['cusum_change_entropy'][i+2] and  
result_df['risk_detection'][i+1] == 0:
             result_df['risk_detection'][i+2] = 1 
        elif result_df['cusum_change_entropy'][i+3] > 0 and
 abs(a) + 15 < result_df['cusum_change_entropy'][i+3] and 
 result_df['risk_detection'][i+2] == 0:
             result_df['risk_detection'][i+3] = 1

    elif result_df['cusum_change_entropy'][i] > 0:
        a = result_df['cusum_change_entropy'][i]
        if result_df['cusum_change_entropy'][i+1] < 0 and
 a > abs(result_df['cusum_change_entropy'][i+1]) + 8 and 
result_df['risk_detection'][i] == 0:
            result_df['risk_detection'][i+1] = 2
        elif result_df['cusum_change_entropy'][i+2] < 0 and
 a > abs(result_df['cusum_change_entropy'][i+2]) + 8 and 
result_df['risk_detection'][i+1] == 0:
            result_df['risk_detection'][i+2] = 2
        elif result_df['cusum_change_entropy'][i+3] < 0 and
 a > abs(result_df['cusum_change_entropy'][i+3]) + 8 and 
result_df['risk_detection'][i+2] == 0:
            result_df['risk_detection'][i+3] = 2

## 실제로 폭등 or 폭락 하지 않은 경우 그 다음 날 2차적 폭등 or 폭락 예상
for i in range(len(result_df)-1):
    if result_df['risk_detection'][i] == 1:
        if abs(result_df['change_close'][i+1]) < 2500:
            result_df['risk_detection'][i+1] = 12     ## 1-2
    if result_df['risk_detection'][i] == 2:
        if abs(result_df['change_close'][i+1]) < 2500:
            result_df['risk_detection'][i+1] = 22     ## 2-2


result_df